In [1]:
import pandas as pd
import sqlite3
from pandasql import sqldf
pysqldf = lambda q:sqldf(q, globals()) 
from sqlite3 import Error

In [2]:
class Database:
    import pandas as pd
    def __init__(self):
        
        """
        input: database path
        """
        self.conn =None
        self.c = None
        self.db_file = None
        self.create_table_Ipl_Venue = None
        self.create_table_ipl_ball_by_ball = None
        self.create_table_match = None
        self.df = None
        self.df1 = None
        self.df2 = None
        self.df_IPL_Bal = pd.read_csv("ipl_ball_by_ball.csv")
        self.df_IPL_Match =  pd.read_csv("IPL_Match.csv")
        self.df_IPL_Venue =  pd.read_csv("IPL_Venue.csv")
        self.df_IPL_Bal_ = None
    def crate_connection(self):
        
        """
        create a database connection to the SQLite database
        input file: database file
        return : Connection object or None
        """
        self.db_file = r"D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db"
        try: 
            #db_file = self.db_file
            print("Database Path:",self.db_file)
            self.conn = sqlite3.connect(self.db_file)
            print("Connected to database....")
            return self.conn
        
        except Exception as e:
            print("Error",e)
            
        return self.conn
    
    def cexcute_table_query(self,crate_sql_table):
        """ create a table from the create_sql statement
            param create_sql_table: a CREATE TABLE statement
            return: Status
        """
        conn=self.crate_connection()
        try:
            self.c = conn.cursor()
            
            self.c.execute(crate_sql_table)
            conn.commit()
            print("Task done....")
            
        except Exception as e:
            print("Error",e)
            
    def create_schema(self):
        """
        Database schema design according to csv data
        """
        create_table_Ipl_Venue = """ CREATE TABLE IF NOT EXISTS IPL_Venue(
                                    venue_id integer PRIMARY KEY,
                                    venue varchar(150)  NULL,
                                    city varchar(150) NULL,

                                    );"""

        create_table_ipl_ball_by_ball = """ CREATE TABLE IF NOT EXISTS IPL_Ball_By_Ball (
                                            match_id integer PRIMARY KEY,
                                            inning integer NULL,
                                            overs integer,
                                            ball integer,
                                            batsman varchar(150),
                                            non_striker varchar(150),
                                            bowler varchar(150),
                                            batsman_runs integer,
                                            extra_runs integer,
                                            total_runs integer,
                                            non_boundary integer,
                                            is_wicket integer,
                                            dismissal_kind varchar(150) NULL,
                                            player_dismissed varchar(150) NULL,
                                            fielder varchar(150) NULL,
                                            extras_type varchar(150) NULL,
                                            batting_team varchar(150) NULL,
                                            bowling_team varchar(150) NULL,
                                        ); """

        create_table_match = """CREATE TABLE IF NOT EXISTS IPL_Match (
                                        match_id integer,
                                        date text NULL,
                                        player_of_match varchar(150),
                                        venue_id integer PRIMARY KEY NOT NULL,
                                        neutral_venue integer NULL,
                                        team1 varchar(150)  NULL,
                                        team2 varchar(150) NULL,
                                        toss_winner varchar(150) NULL,
                                        toss_decision varchar(150) NULL,
                                        winner varchar(150) NULL,
                                        result varchar(150) NUll,
                                        result_margin integer NULL,
                                        eliminator varchar(50),
                                        method varchar(50),
                                        umpire1 varchar(150),
                                        umpire2 varchar(150),
                                    );"""
        # function call
        conn = self.crate_connection()
        # create tables
        if conn is not None:
            self.cexcute_table_query(create_table_Ipl_Venue)
            self.cexcute_table_query(create_table_match)
            self.cexcute_table_query(create_table_Ipl_Venue)
        else:
            print("Error!Database connection error.")

    def load_data_into_sqllit(self):
        """
        load csv data in IPL.db database
        """
        try:
            self.df = pd.read_csv("ipl_ball_by_ball.csv")
            self.df1 = pd.read_csv("IPL_Match.csv")
            self.df2 = pd.read_csv("IPL_Venue.csv")
            # function call
            conn = self.crate_connection()
            # create tables
            if conn is not None:
                # load the data in IPL.db database
                self.df.to_sql('IPL_Ball_By_Ball', conn, if_exists='append', index = False)
                self.df1.to_sql('IPL_Match', conn, if_exists='append', index = False)
                self.df2.to_sql('IPL_Venue', conn, if_exists='append', index = False)
                print("Done......")
            else:
                print("Error!Database connection error.")
        except Exception as e:
            print("Error",e)
    def get_query1_result(self):
        """ 
        Q1. Write a query to find the highest extra runs given by a team in a match
        """
        try:
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute("""
            SELECT bowling_team,Max(extra_runs) as Highest_Extra_runs
            FROM IPL_Ball_By_Ball
            WHERE bowling_team != 'NA'
            GROUP BY bowling_team
            ORDER BY Highest_Extra_runs DESC;""")
            return pd.DataFrame(c.fetchall(),columns=['bowling_team','Highest_Extra_runs'])
        
        except Exception as e:
            print("Error",e)
            
    def get_query2_result(self):
        """ 
        Q2. Write a query to find the Leading wicket taker in the IPL?
        Note: A wicket won't be accounted for the bowler, if the dismissal is run out, 
        obstructing the field.
        """
        try:
            query2 = """
            SELECT bowler as Bowler,count(is_wicket) as Count_wicket
            FROM IPL_Ball_By_Ball
            WHERE dismissal_kind !='NA' and dismissal_kind != 'obstructing the field' and dismissal_kind !='run out'
            GROUP BY Bowler
            ORDER BY Count_wicket DESC
            """
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute(query2)
            
            return pd.DataFrame(c.fetchall(),columns=['Bowler','Count_Wicket'])
        
        except Exception as e:
            print("Error",e)
            
    def get_query3_result(self):
        """ 
        Q3. Write a query to return a report for highest run scorer in matches which were 
                affected by Duckworth-Lewis’s method (D/L method).
        """
        
        try:
            query3 = """
            SELECT batsman, method, sum(total_runs) AS Max_Run
            FROM IPL_Ball_By_Ball as df
            INNER JOIN IPL_Match as df1 ON df.match_id=df1.match_id
            WHERE method='D/L'
            GROUP BY batsman, method
            ORDER BY Max_Run DESC
            """
            
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute(query3)
            
            return pd.DataFrame(c.fetchall(),columns=['Batsman','Method','Max_Run'])
        except Exception as e:
            print("Error",e)
            
    def get_query4_result(self):
        """Write a query to return a report for highest strike rate by a batsman in 
            powerplay (1-6 overs)
            Note: strike rate = (Total Runs scored/Total balls faced by player) *100, Make 
            sure that balls faced by player should be legal delivery (not wide balls or no)
            balls
        """
        
        try:
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute(""" SELECT batsman, ((sum(batsman_runs)/count(batsman_runs))*100) as Strike_Rate
                    FROM IPL_Ball_By_Ball
                    WHERE overs between 1 and 6 and extra_runs=0
                    GROUP BY batsman
                    ORDER BY Strike_Rate DESC """)
            return pd.DataFrame(c.fetchall(), columns=['Batsman','Strike Rate']).iloc[1:,0:]
        except Exception as e:
            print("Error:",e)
            
    def get_query5_result(self):
        """
        Q5 Write a query to return a report for highest extra runs in a venue (stadium, city).
        
        """
        
        try: 
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute(""" SELECT venue,city, MAX(extra_runs) as Max_Extra_Runs
                    FROM IPL_Ball_By_Ball as df
                    INNER JOIN IPL_Match as df1 ON df.match_id=df1.match_id
                    INNER JOIN IPL_Venue as df2 ON df1.venue_id = df2.venue_id
                    GROUP BY venue,city
                    ORDER BY Max_Extra_Runs DESC;
                    """)
            return pd.DataFrame(c.fetchall(),columns=[["venue","city","max_extra_euns"]])
        except Exception as e:
            print("Error",e)
    def get_query6_result(self):   
        
        """
        Q6. Write a query to return a report for the cricketers with the most number of 
        players of the match award in neutral venues.
            """
        try: 
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute("""SELECT player_of_match,COUNT(player_of_match) as Most_player_of_match
                    FROM IPL_Match
                    WHERE neutral_venue=1
                    GROUP BY player_of_match
                    ORDER BY  Most_player_of_match DESC
                    """)
            return pd.DataFrame(c.fetchall(),columns=['player_of_match','Most player_of_match'])
        except Exception as e:
            print("Error",e)
            
    def get_query7_result(self):
        """
        Q7. Write a query to get a list of top 10 players with the highest batting averager 
        of times they have been out (Make sure to include run outs (on nonstriker end) as valid out while calculating average)
        """
        try:
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute(""" SELECT batsman,SUM(batsman_runs)/SUM(CASE WHEN is_wicket=1 and (player_dismissed = non_striker or player_dismissed = batsman) THEN 1 ELSE 0 END) as average 
                    FROM  IPL_Ball_By_Ball 
                    group by batsman 
                    order by average desc 
                    limit 10;""")
            return pd.DataFrame(c.fetchall(),columns=['Batsman','Average'])
        except Exception as e:
            print("Error",e)
            
    def get_query8_result(self):
        """
        Q8. Write a query to find out who has officiated (as an umpire) the most number of matches in IPL.

        """
        try:
            umpire1 = self.df_IPL_Match['umpire1'].value_counts()
            umpire2 = self.df_IPL_Match['umpire2'].value_counts()
            umpires_df = pd.concat([umpire1, umpire2], axis=1)

            umpire = umpires_df.sum(axis=1)

            umpire_df = pd.DataFrame(columns=["umpire", "matches"])

            for items in umpire.iteritems():
                temp_df = pd.DataFrame({
                    'umpire':[items[0]],
                    'matches':[items[1]]
                })
                umpire_df= umpire_df.append(temp_df, ignore_index=True) 
            umpire_df.sort_values('matches', ascending=False,inplace=True)
            return umpire_df
        except Exception as e:
            print("Error",e)
    def get_query9_result(self):
        """
        Q9. Find venue details of the match where V Kohli scored his highest individual runs in IPL.
        """
        try: 
            curr = self.crate_connection()
            c=curr.cursor()
            c.execute(""" SELECT batsman,venue,city,date, SUM("batsman_runs") as Max_Run
                    FROM IPL_Ball_By_Ball as df
                    INNER JOIN IPL_Match as df1 ON df.match_id=df1.match_id
                    INNER JOIN IPL_Venue as df2 ON df1.venue_id = df2.venue_id
                    WHERE batsman='V Kohli'
                    GROUP BY batsman,venue,city,date
                    ORDER BY Max_Run DESC;
                    """)
            return pd.DataFrame(c.fetchall(),columns=[["batsman","venue","city","date","Max_Run"]])
        except Exception as e:
            print("Error",e)

In [3]:
db = Database()

In [4]:
db.get_query1_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,bowling_team,Highest_Extra_runs
0,Mumbai Indians,7
1,Sunrisers Hyderabad,5
2,Royal Challengers Bangalore,5
3,Rising Pune Supergiants,5
4,Rising Pune Supergiant,5
5,Rajasthan Royals,5
6,Pune Warriors,5
7,Kolkata Knight Riders,5
8,Kings XI Punjab,5
9,Gujarat Lions,5


In [5]:
db.get_query2_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,Bowler,Count_Wicket
0,SL Malinga,170
1,A Mishra,160
2,PP Chawla,156
3,DJ Bravo,153
4,Harbhajan Singh,150
...,...,...
355,AJ Finch,1
356,AC Gilchrist,1
357,AA Noffke,1
358,AA Jhunjhunwala,1


In [6]:
db.get_query3_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,Batsman,Method,Max_Run
0,CH Gayle,D/L,260
1,DA Warner,D/L,199
2,V Kohli,D/L,176
3,Yuvraj Singh,D/L,157
4,S Dhawan,D/L,152
...,...,...,...
106,DJ Hussey,D/L,0
107,F Behardien,D/L,0
108,KJ Abbott,D/L,0
109,MF Maharoof,D/L,0


In [7]:
db.get_query4_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,Batsman,Strike Rate
1,CH Morris,200
2,AD Russell,200
3,AD Mascarenhas,200
4,YV Takawale,100
5,YK Pathan,100
...,...,...
285,AA Jhunjhunwala,0
286,AA Bilakhia,0
287,A Mishra,0
288,A Chopra,0


In [8]:
db.get_query5_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,venue,city,max_extra_euns
0,Sharjah Cricket Stadium,Sharjah,7
1,Brabourne Stadium,Mumbai,5
2,Dr DY Patil Sports Academy,Mumbai,5
3,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...,Visakhapatnam,5
4,Dubai International Cricket Stadium,Dubai,5
5,Eden Gardens,Kolkata,5
6,Feroz Shah Kotla,Delhi,5
7,Green Park,Kanpur,5
8,Himachal Pradesh Cricket Association Stadium,Dharamsala,5
9,Holkar Cricket Stadium,Indore,5


In [9]:
db.get_query5_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,venue,city,max_extra_euns
0,Sharjah Cricket Stadium,Sharjah,7
1,Brabourne Stadium,Mumbai,5
2,Dr DY Patil Sports Academy,Mumbai,5
3,Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket St...,Visakhapatnam,5
4,Dubai International Cricket Stadium,Dubai,5
5,Eden Gardens,Kolkata,5
6,Feroz Shah Kotla,Delhi,5
7,Green Park,Kanpur,5
8,Himachal Pradesh Cricket Association Stadium,Dharamsala,5
9,Holkar Cricket Stadium,Indore,5


In [10]:
db.get_query6_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,player_of_match,Most player_of_match
0,YK Pathan,3
1,JP Duminy,3
2,JH Kallis,3
3,GJ Maxwell,3
4,Yuvraj Singh,2
5,Sandeep Sharma,2
6,SR Tendulkar,2
7,SK Raina,2
8,RG Sharma,2
9,ML Hayden,2


In [11]:
db.get_query7_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,Batsman,Average
0,Iqbal Abdulla,88
1,RD Gaikwad,68
2,MN van Wyk,55
3,PD Collingwood,50
4,LH Ferguson,44
5,HM Amla,44
6,KL Rahul,42
7,AB de Villiers,42
8,ML Hayden,41
9,JP Duminy,41


In [12]:
db.get_query8_result()

,umpire,matches
5,S Ravi,121.0
0,HDPK Dharmasena,94.0
1,AK Chaudhary,87.0
11,C Shamshuddin,82.0
3,M Erasmus,65.0
29,CK Nandan,57.0
14,Nitin Menon,57.0
47,SJA Taufel,55.0
2,Asad Rauf,51.0
15,VA Kulkarni,50.0


In [13]:
db.get_query9_result()

Database Path: D:\Data Engg interview\DataGrokr\ipl_dataset\IPL.db
Connected to database....


,batsman,venue,city,date,Max_Run
0,V Kohli,M.Chinnaswamy Stadium,Bengaluru,2016-05-18,113
1,V Kohli,M.Chinnaswamy Stadium,Bengaluru,2016-05-14,109
2,V Kohli,M.Chinnaswamy Stadium,Bengaluru,2016-05-07,108
3,V Kohli,Eden Gardens,Kolkata,2019-04-19,100
4,V Kohli,Saurashtra Cricket Association Stadium,Rajkot,2016-04-24,100
...,...,...,...,...,...
179,V Kohli,Eden Gardens,Kolkata,2017-04-23,0
180,V Kohli,M.Chinnaswamy Stadium,Bengaluru,2008-05-28,0
181,V Kohli,M.Chinnaswamy Stadium,Bengaluru,2014-05-04,0
182,V Kohli,M.Chinnaswamy Stadium,Bengaluru,2014-05-09,0
